# 네이버 데이터랩 검색어 500개 수집 프로그램 (사용자 코드 입력 버전)

## 프로그램 설명
네이버 데이터랩에서 사용자가 입력한 코드를 기반으로 500개의 인기 검색어를 자동으로 수집하는 프로그램입니다.

## 주요 기능
- 사용자로부터 코드 입력 받기
- 네이버 데이터랩 인기 검색어 500개 자동 수집
- 엑셀 파일 자동 저장
- 자동 페이지네이션 처리 (25페이지)

## 사용 방법
1. 프로그램 실행
2. 사용자 코드 입력
3. 저장할 엑셀 파일 기본 이름 입력
4. 자동으로 500개 검색어 수집 및 엑셀 저장

## 필요 라이브러리
- selenium: 웹 크롤링
- openpyxl: 엑셀 파일 처리

## 주의사항
- 네이버 데이터랩 페이지 구조 변경 시 수정이 필요할 수 있습니다
- 사용자 코드 형식이 맞아야 합니다

In [2]:
import requests
import hmac
import hashlib
import base64
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font
import os
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import logging
import subprocess
from tqdm import tqdm

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def generate_signature(timestamp, method, uri, secret_key):
    try:
        message = f"{timestamp}.{method}.{uri}"
        signature = hmac.new(secret_key.encode(), message.encode(), hashlib.sha256).digest()
        signature_b64 = base64.b64encode(signature).decode()
        return signature_b64
    except Exception as e:
        logging.error(f"Error generating signature: {e}")
        return None

def get_naver_search_volume(api_key, secret_key, customer_id, keyword):
    max_retries = 5
    retry_delay = 5  # seconds

    for attempt in range(max_retries):
        try:
            timestamp = str(int(time.time() * 1000))
            method = "GET"
            uri = "/keywordstool"
            signature = generate_signature(timestamp, method, uri, secret_key)
            
            if not signature:
                return None

            url = "https://api.naver.com" + uri
            headers = {
                "X-API-KEY": api_key,
                "X-API-SECRET": secret_key,
                "X-CUSTOMER": customer_id,
                "X-Timestamp": timestamp,
                "X-Signature": signature,
                "Content-Type": "application/json",
                "Accept": "*/*"
            }
            params = {
                "hintKeywords": keyword,
                "showDetail": "1"
            }
            
            response = requests.get(url, headers=headers, params=params)
            
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:
                logging.warning(f"429 Too Many Requests. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                logging.error(f"Error: {response.status_code}, {response.text}")
                return None
        except Exception as e:
            logging.error(f"Error fetching search volume: {e}")
            return None
    
    logging.error("Max retries reached. Unable to fetch search volume.")
    return None

def write_to_excel(results, search_date, file_path="search_volume.xlsx"):
    try:
        wb = Workbook()
        ws = wb.active
        ws.title = "Search Volume"
        
        # 한글 헤더 추가
        headers = ["순위", "검색어", "월간총검색량", "검색 날짜", "쿠팡 링크", "네이버 쇼핑 링크"]
        ws.append(headers)
        
        # 링크 스타일 설정
        link_font = Font(color="0000FF", underline="single")
        
        # 결과 데이터 추가
        for rank, (keyword, total_search_volume) in enumerate(results.items(), start=1):
            coupang_link = f"https://www.coupang.com/np/search?component=&q={keyword}&channel=user"
            naver_link = f"https://search.shopping.naver.com/search/all?query={keyword}"
            ws.append([
                rank, 
                keyword, 
                total_search_volume, 
                search_date, 
                f'=HYPERLINK("{coupang_link}", "쿠팡 링크")', 
                f'=HYPERLINK("{naver_link}", "네이버 쇼핑 링크")'
            ])
            ws[f"E{rank+1}"].font = link_font
            ws[f"F{rank+1}"].font = link_font
        
        # 열 너비 자동 조정
        for col in ws.columns:
            max_length = 0
            column = col[0].column_letter  # Get the column name
            for cell in col:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2)
            ws.column_dimensions[column].width = adjusted_width
        
        wb.save(file_path)
        logging.info(f"Data saved to {file_path}")
    except Exception as e:
        logging.error(f"Error writing to Excel: {e}")

def select_dropdown_option(wait, dropdown_xpath, option_xpath):
    dropdown = wait.until(EC.element_to_be_clickable((By.XPATH, dropdown_xpath)))
    dropdown.click()
    option = wait.until(EC.element_to_be_clickable((By.XPATH, option_xpath)))
    option.click()
    time.sleep(1)  # 드롭다운 선택 후 대기

def main():
    api_key = "0100000000f7219646b1189747046631aa4d35e62954247e750a216e37f7072b5174cbe5c2"
    secret_key = "AQAAAAD3IZZGsRiXRwRmMapNNeYpncLjvqyOrRpn2cZDTV49Ug=="
    customer_id = "373124"

    # 현재 날짜를 yyyymmdd 형식으로 얻기
    current_date = datetime.now().strftime("%Y%m%d")
    search_date = datetime.now().strftime("%Y-%m-%d")  # 검색 날짜를 yyyy-mm-dd 형식으로 얻기

    # 사용자로부터 저장할 Excel 파일 기본 이름 입력받기
    base_file_name = input("저장할 Excel 파일의 기본 이름을 입력하세요 (확장자 제외): ")

    # 입력받은 파일 이름에 오늘 날짜 추가
    excel_file_name = f"{base_file_name}_{current_date}.xlsx"

    # 현재 작업 중인 폴더의 경로를 얻기
    current_working_directory = os.getcwd()
    file_path = os.path.join(current_working_directory, excel_file_name)  # 파일 경로 결합

    # 검색할 페이지 수 입력받기
    num_pages = int(input("검색할 페이지 수를 입력하세요: "))

    # WebDriver 설정(Chrome 사용)
    try:
        driver = webdriver.Chrome()
    except Exception as e:
        logging.error(f"Error initializing WebDriver: {e}")
        return

    # Excel 설정
    wb = Workbook()
    ws = wb.create_sheet('검색어순위')
    wb.remove(wb['Sheet'])  # 기본 생성된 시트 제거
    ws.append(['순위', '인기검색어', '검색 날짜'])

    try:
        # 네이버 데이터랩 쇼핑 인사이트 페이지로 이동
        driver.get("https://datalab.naver.com/shoppingInsight/sCategory.naver")

        # 필요한 요소가 로드될 때까지 대기하는 WebDriverWait 인스턴스 생성
        wait = WebDriverWait(driver, 20)

        # 드롭다운 옵션 선택을 위한 사용자 입력 받기
        first_option = int(input("첫 번째 드롭다운 옵션을 입력하세요 (필수): "))
        second_option = input("두 번째 드롭다운 옵션을 입력하세요 (선택, 없으면 0 또는 공백): ")
        third_option = input("세 번째 드롭다운 옵션을 입력하세요 (선택, 없으면 0 또는 공백): ")
        fourth_option = input("네 번째 드롭다운 옵션을 입력하세요 (선택, 없으면 0 또는 공백): ")

        # 드롭다운 및 옵션 선택
        select_dropdown_option(wait, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/span', f'//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/ul/li[{first_option}]/a')

        if second_option and second_option.strip() and second_option != '0':
            try:
                select_dropdown_option(wait, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[2]/span', f'//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[2]/ul/li[{int(second_option)}]/a')
            except Exception as e:
                logging.error(f"Error occurred while selecting the second option: {e}")
        else:
            second_option = None

        if third_option and third_option.strip() and third_option != '0' and second_option:
            try:
                select_dropdown_option(wait, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/span', f'//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li[{int(third_option)}]/a')
            except Exception as e:
                logging.error(f"Error occurred while selecting the third option: {e}")
        else:
            third_option = None

        if fourth_option and fourth_option.strip() and fourth_option != '0' and third_option:
            try:
                select_dropdown_option(wait, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[4]/span', f'//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[4]/ul/li[{int(fourth_option)}]/a')
            except Exception as e:
                logging.error(f"Error occurred while selecting the fourth option: {e}")

        # 기기별, 성별, 연령 전체 선택
        driver.find_element(By.XPATH, '//*[@id="18_device_0"]').click()
        driver.find_element(By.XPATH, '//*[@id="19_gender_0"]').click()

        # 연령대 선택
        age_checkbox = driver.find_element(By.XPATH, '//*[@id="20_age_0"]')
        driver.execute_script("arguments[0].scrollIntoView(true);", age_checkbox)
        age_checkbox.click()

        time.sleep(2)  # 필요한 요소 로딩 대기

        # '조회하기' 버튼 클릭
        search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/a')))
        search_button.click()

        # 결과 스크래핑 및 Excel 파일 저장
        keywords = []
        for i in range(num_pages):  # 사용자로부터 입력받은 페이지 수만큼 반복
            # 페이지 로딩 대기
            time.sleep(1)
            
            for j in range(1, 21):  # 한 페이지당 최대 20개 아이템
                try:
                    path = f'//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[1]/ul/li[{j}]/a'
                    result = driver.find_element(By.XPATH, path).text
                    result_data = result.split('\n')
                    result_data[0] = int(result_data[0])  # 순위를 숫자 형식으로 변환
                    result_data.append(search_date)  # 검색 날짜 추가
                    logging.info(result_data)
                    ws.append(result_data)
                    keywords.append(result_data[1])  # 인기검색어 수집
                except Exception as e:
                    logging.error(f"Error occurred while scraping data: {e}")
            
            # 다음 페이지로 넘어가는 버튼 클릭 로직 (실제 페이지 구조에 따라 달라질 수 있음)
            try:
                next_button = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/a[2]')
                next_button.click()
                time.sleep(1)  # 페이지 로딩 대기
            except Exception as e:
                logging.error(f"Error occurred while clicking next button: {e}")
                break

        # 파일 저장
        try:
            wb.save(file_path)
            wb.close()
            logging.info(f"'{excel_file_name}' 파일이 현재 폴더에 저장되었습니다.")
        except Exception as e:
            logging.error(f"Error occurred while saving the file: {e}")

    except Exception as e:
        logging.error(f"An error occurred during scraping: {e}")
    finally:
        driver.quit()

    # 검색어에 대한 검색량 수집
    logging.info("검색량 수집이 수분 이상 걸릴 수 있으니 잠시만 기다려 주세요.")
    results = {}
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(get_naver_search_volume, api_key, secret_key, customer_id, keyword): keyword for keyword in keywords}
        
        # 진행 상황을 퍼센트로 표시
        for future in tqdm(futures, desc="Fetching search volumes", unit="keyword", ncols=100):
            keyword = futures[future]
            try:
                data = future.result()
                if data and "keywordList" in data and len(data["keywordList"]) > 0:
                    item = data["keywordList"][0]
                    monthlyPcQcCnt = item.get("monthlyPcQcCnt", 0)
                    monthlyMobileQcCnt = item.get("monthlyMobileQcCnt", 0)

                    # '< 10' 처리
                    if isinstance(monthlyPcQcCnt, str) and '<' in monthlyPcQcCnt:
                        monthlyPcQcCnt = 5  # '< 10'인 경우, 5로 설정 (임의 값)
                    else:
                        monthlyPcQcCnt = int(monthlyPcQcCnt)

                    if isinstance(monthlyMobileQcCnt, str) and '<' in monthlyMobileQcCnt:
                        monthlyMobileQcCnt = 5  # '< 10'인 경우, 5로 설정 (임의 값)
                    else:
                        monthlyMobileQcCnt = int(monthlyMobileQcCnt)

                    total_search_volume = monthlyPcQcCnt + monthlyMobileQcCnt
                    results[keyword] = total_search_volume
            except Exception as e:
                logging.error(f"Error occurred while fetching search volume for {keyword}: {e}")

            # 요청 간의 딜레이 추가
            time.sleep(2)  # 딜레이를 2초로 설정

    write_to_excel(results, search_date, file_path)

    # 저장된 파일 열기
    try:
        subprocess.Popen(['start', file_path], shell=True)
    except Exception as e:
        logging.error(f"Error occurred while opening the file: {e}")

if __name__ == "__main__":
    main()


저장할 Excel 파일의 기본 이름을 입력하세요 (확장자 제외): 뜨개도구
검색할 페이지 수를 입력하세요: 25
첫 번째 드롭다운 옵션을 입력하세요 (필수): 5
두 번째 드롭다운 옵션을 입력하세요 (선택, 없으면 0 또는 공백): 16
세 번째 드롭다운 옵션을 입력하세요 (선택, 없으면 0 또는 공백): 1
네 번째 드롭다운 옵션을 입력하세요 (선택, 없으면 0 또는 공백): 2


2024-06-12 09:59:50,283 - INFO - [1, '코바늘', '2024-06-12']
2024-06-12 09:59:50,294 - INFO - [2, '코바늘세트', '2024-06-12']
2024-06-12 09:59:50,306 - INFO - [3, '튤립코바늘세트', '2024-06-12']
2024-06-12 09:59:50,316 - INFO - [4, '튤립코바늘', '2024-06-12']
2024-06-12 09:59:50,327 - INFO - [5, '대바늘', '2024-06-12']
2024-06-12 09:59:50,339 - INFO - [6, '에티모코바늘', '2024-06-12']
2024-06-12 09:59:50,349 - INFO - [7, '에티모코바늘세트', '2024-06-12']
2024-06-12 09:59:50,361 - INFO - [8, '튤립에티모코바늘', '2024-06-12']
2024-06-12 09:59:50,375 - INFO - [9, '니팅링', '2024-06-12']
2024-06-12 09:59:50,388 - INFO - [10, '크로바코바늘', '2024-06-12']
2024-06-12 09:59:50,398 - INFO - [11, '치아오구', '2024-06-12']
2024-06-12 09:59:50,410 - INFO - [12, '코바늘케이스', '2024-06-12']
2024-06-12 09:59:50,423 - INFO - [13, '대바늘세트', '2024-06-12']
2024-06-12 09:59:50,434 - INFO - [14, '레이스코바늘', '2024-06-12']
2024-06-12 09:59:50,445 - INFO - [15, '크로바코바늘세트', '2024-06-12']
2024-06-12 09:59:50,457 - INFO - [16, '스티치홀더', '2024-06-12']
2024-06-12 09:59:50,469 -

2024-06-12 10:00:03,745 - INFO - [133, '코바늘셋트', '2024-06-12']
2024-06-12 10:00:03,755 - INFO - [134, '니트프로데님', '2024-06-12']
2024-06-12 10:00:03,763 - INFO - [135, '니트프로세트', '2024-06-12']
2024-06-12 10:00:03,772 - INFO - [136, '스텐대바늘', '2024-06-12']
2024-06-12 10:00:03,782 - INFO - [137, '실리콘코바늘세트', '2024-06-12']
2024-06-12 10:00:03,795 - INFO - [138, '긴코바늘', '2024-06-12']
2024-06-12 10:00:03,805 - INFO - [139, '마인드풀대바늘', '2024-06-12']
2024-06-12 10:00:03,814 - INFO - [140, '양쪽코바늘', '2024-06-12']
2024-06-12 10:00:05,857 - INFO - [141, '일제코바늘', '2024-06-12']
2024-06-12 10:00:05,868 - INFO - [142, '튤립레드코바늘', '2024-06-12']
2024-06-12 10:00:05,876 - INFO - [143, '아디노블', '2024-06-12']
2024-06-12 10:00:05,885 - INFO - [144, '아디줄바늘', '2024-06-12']
2024-06-12 10:00:05,894 - INFO - [145, '타쿠미', '2024-06-12']
2024-06-12 10:00:05,902 - INFO - [146, '튤립로즈코바늘', '2024-06-12']
2024-06-12 10:00:05,911 - INFO - [147, '나무코바늘', '2024-06-12']
2024-06-12 10:00:05,919 - INFO - [148, '뜨개실', '2024-06-12']
202

2024-06-12 10:00:19,141 - INFO - [263, '히야히야스틸', '2024-06-12']
2024-06-12 10:00:19,149 - INFO - [264, '7mm코바늘', '2024-06-12']
2024-06-12 10:00:19,158 - INFO - [265, '니나니팅', '2024-06-12']
2024-06-12 10:00:19,167 - INFO - [266, '독일코바늘', '2024-06-12']
2024-06-12 10:00:19,175 - INFO - [267, '레드코바늘', '2024-06-12']
2024-06-12 10:00:19,184 - INFO - [268, '레이스용코바늘세트', '2024-06-12']
2024-06-12 10:00:19,193 - INFO - [269, '마인드풀케이블', '2024-06-12']
2024-06-12 10:00:19,201 - INFO - [270, '바늘이야기니팅링', '2024-06-12']
2024-06-12 10:00:19,210 - INFO - [271, '아뮤레세트', '2024-06-12']
2024-06-12 10:00:19,218 - INFO - [272, '앵콜스', '2024-06-12']
2024-06-12 10:00:19,227 - INFO - [273, '코바늘골무', '2024-06-12']
2024-06-12 10:00:19,239 - INFO - [274, '코바늘튤립레드', '2024-06-12']
2024-06-12 10:00:19,248 - INFO - [275, '코바늘후크', '2024-06-12']
2024-06-12 10:00:19,258 - INFO - [276, '튤립모사용바늘', '2024-06-12']
2024-06-12 10:00:19,267 - INFO - [277, '튤립코바늘에티모', '2024-06-12']
2024-06-12 10:00:19,276 - INFO - [278, '흑단바늘', '2024-06

2024-06-12 10:00:32,425 - INFO - [392, '나무손잡이코바늘', '2024-06-12']
2024-06-12 10:00:32,433 - INFO - [393, '니트프로대바늘12mm', '2024-06-12']
2024-06-12 10:00:32,441 - INFO - [394, '니트프로바늘', '2024-06-12']
2024-06-12 10:00:32,450 - INFO - [395, '니트프로심포니우드', '2024-06-12']
2024-06-12 10:00:32,459 - INFO - [396, '대바늘보관케이스', '2024-06-12']
2024-06-12 10:00:32,468 - INFO - [397, '라이키바늘', '2024-06-12']
2024-06-12 10:00:32,477 - INFO - [398, '마벨로타', '2024-06-12']
2024-06-12 10:00:32,486 - INFO - [399, '아디숏팁', '2024-06-12']
2024-06-12 10:00:32,494 - INFO - [400, '아뮤레코바늘세트', '2024-06-12']
2024-06-12 10:00:34,524 - INFO - [401, '에티모코바늘직구', '2024-06-12']
2024-06-12 10:00:34,534 - INFO - [402, '이녹스대바늘', '2024-06-12']
2024-06-12 10:00:34,543 - INFO - [403, '자단바늘', '2024-06-12']
2024-06-12 10:00:34,551 - INFO - [404, '줄바늘12mm', '2024-06-12']
2024-06-12 10:00:34,560 - INFO - [405, '진저스페셜', '2024-06-12']
2024-06-12 10:00:34,568 - INFO - [406, '치아오구레드케이블', '2024-06-12']
2024-06-12 10:00:34,576 - INFO - [407, '코바늘